### Άσκηση ταξινόμησης

Για τον πειραματισμό μας  θα χρησιμοποιήσουμε ως dataset το [Pima-Indians-diabetes](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database). Θα φορτώσουμε το dataset και θα το χωρίσουμε σε data και σε labels.

In [2]:
from IPython.display import Image
from IPython.display import display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# inline plotting instead of popping out
%matplotlib inline


# e.g., plot_decision_regions()
import os, sys
module_path = os.path.abspath(os.path.join('.'))
sys.path.append(module_path)


# Load CSV using Pandas from URL

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)
print(data.shape)

# separate array into input and output components
# X: Features Y: Labels

y = data['class']
X = data.drop('class', axis=1)

X.shape, y.shape

(768, 9)


((768, 8), (768,))

1. Χωρίστε τα data kai ta labels σε training (80%) και σε testing set (20%).

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

2.  Έπειτα χωρίστε το training set σε 70% και σε 30% όπου το 30% θα το χρησιμοποιήσουμε για validation.

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

3. Για διαφορετικές υπερπαραμέτρους (n_neighbors = 1, 2, 10) να εκπαιδεύσετε έναν k-nearest neighbors classifier, χρησιμοποιώντας το training set με το 
validation set  με 5 fold cross validation και τυπώστε τα αποτελέσματα.

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

best_k, best_score = -1, -1
clfs = {}

for k in [1, 2, 10]: # experiment different hyperparameter
    knn1 = KNeighborsClassifier(n_neighbors=k)
    knn1.fit(X_train, y_train)
    # K-Fold CV
    scores = cross_val_score(knn1, X_train, y_train, cv=5)
    print('[%d-NN]\nValidation accuracy: %.3f %s' % (k, scores.mean(), scores))
    if scores.mean() > best_score:
        best_k, best_score = k, scores.mean()
    clfs[k] = knn1

[1-NN]
Validation accuracy: 0.683 [0.73333333 0.64       0.72       0.69333333 0.62666667]
[2-NN]
Validation accuracy: 0.696 [0.65333333 0.69333333 0.73333333 0.73333333 0.66666667]
[10-NN]
Validation accuracy: 0.707 [0.70666667 0.77333333 0.70666667 0.68       0.66666667]


4. Επιλέξτε το καλύτερο αποτέλεσμα και εφαρμόστε αυτό το μοντέλο στα δεδομένα ελέγχου (test set).

In [23]:
best_clf = clfs[best_k]
best_clf.fit(X_train, y_train)

# performance reporting
y_pred = best_clf.predict(X_test)
print('Test accuracy: %.2f (n_neighbors=%d selected by 5-fold CV)' % 
      (accuracy_score(y_test, y_pred), best_k))
cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)

Test accuracy: 0.73 (n_neighbors=10 selected by 5-fold CV)
[[137  20]
 [ 42  32]]


5. Eκπαιδεύσετε έναν  Gaussian Naive Bayes  classifier, χρησιμοποιώντας το training set με το validation set με 5 fold cross validation και τυπώστε τα αποτελέσματα. 

In [17]:
from sklearn.naive_bayes import GaussianNB

nbc = GaussianNB()
nbc.fit(X_train,y_train)
scores = cross_val_score(nbc, X_train, y_train, cv=5)
print(scores, scores.mean())

[0.76       0.81333333 0.70666667 0.74666667 0.68      ] 0.7413333333333333


6. Eφαρμόστε το μοντέλο που εκπαιδεύσατε στα δεδομένα ελέγχου (test set)

In [24]:
y_pred = nbc.predict(X_test)
print(accuracy_score(y_test, y_pred))
cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)

0.7575757575757576
[[137  20]
 [ 36  38]]


7. Εκτυπώστε τον confusion matrix και για τους δύο ταξινομητές που εκπαιδεύσατε και ελέγξατε.
